In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/train.csv
/kaggle/input/tabular-playground-series-mar-2021/test.csv


In [1]:
# importing libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [3]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')

We're preprocessing the data and passing them through a not tuned XGBoost to set a baseline

We define the X and the y to give at our model

In [4]:
X = train.drop(['target', 'id'] , axis=1)
y = train['target']

We split numerical and categorical columns because we will process them differently

In [5]:
cat_columns = list(X.select_dtypes('object').columns)
num_columns = list(X.select_dtypes('float64').columns)

Splitting data in train and validation

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

Creating the column transformer

In [7]:
cat_transformer = OneHotEncoder(handle_unknown='ignore')
num_transformer = QuantileTransformer(n_quantiles=100, output_distribution='normal')

preprocessor = ColumnTransformer(
        transformers=[
            ('num', num_transformer, num_columns),
            ('cat', cat_transformer, cat_columns)
        ])

Creating the model

In [8]:
model = XGBRegressor(tree_method='gpu_hist', n_estimators=1000, learning_rate=0.05, random_state=0)

Creating the pipeline, fitting the model and test on validation

In [9]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [10]:
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  QuantileTransformer(n_quantiles=100,
                                                                      output_distribution='normal'),
                                                  ['cont0', 'cont1', 'cont2',
                                                   'cont3', 'cont4', 'cont5',
                                                   'cont6', 'cont7', 'cont8',
                                                   'cont9', 'cont10']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['cat0', 'cat1', 'cat2',
                                                   'cat3', 'cat4', 'cat5',
                                                   'cat6', 'cat7', 'cat8',
                                            

In [11]:
val_pred = my_pipeline.predict(X_valid)
val_pred

array([ 0.85629314,  0.24161845, -0.0101608 , ...,  0.06532884,
        0.62317485,  0.23201245], dtype=float32)

In [13]:
my_pipeline.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  QuantileTransformer(n_quantiles=100,
                                                                      output_distribution='normal'),
                                                  ['cont0', 'cont1', 'cont2',
                                                   'cont3', 'cont4', 'cont5',
                                                   'cont6', 'cont7', 'cont8',
                                                   'cont9', 'cont10']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['cat0', 'cat1', 'cat2',
                                                   'cat3', 'cat4', 'cat5',
                                                   'cat6', 'cat7', 'cat8',
                                            

In [14]:
X_test = test.drop(['id'], axis=1)
prediction = my_pipeline.predict(X_test)
prediction

ValueError: X has 31 features, but ColumnTransformer is expecting 30 features as input.

Creating submission file

In [ ]:
submission = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')
submission['target'] = prediction
submission = submission.set_index('id')
submission
submission.to_csv('xgboostreg_simple1.csv')